# Package Imports

In [3]:
import pandas as pd
import numpy as np
#import plotly.express as px
import json
import time
import os

from ediblepickle import checkpoint
from urllib.parse import quote
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

pd.options.plotting.backend = "plotly"

In [4]:
np.__version__

'1.20.3'

In [5]:
# Extra code to show dataframes
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Data Importing
Do not assign to imported variables!

In [52]:
# Deprecated imports
ZIPCodeIncome = pd.read_csv(data_dir+"ACSST5Y2020.S1903_data_with_overlays_2022-04-24T152354.csv", 
                            skiprows=1, 
                            converters = {'250,000+':250000}, 
                            usecols=["Geographic Area Name", "Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households"]
                           )


### School Information
Information found in this section with the exception of school_cohort are not broken down by demographics. We can merge these dataframes first by the school key. 

In [6]:
data_dir = "Data Files/"

school_cohort = pd.read_table(data_dir + "cohort2021.txt", 
                              dtype={'CountyCode': str, 'DistrictCode': str, 'SchoolCode': str}, 
                              na_values="*"
                              )

school_info = pd.read_table(data_dir + "pubschls.txt", 
                            dtype={'CDSCode': str}, 
                            #usecols = ['CDSCode', 'StatusType', 'Zip', 'Street', 'Latitude', 'Longitude']
                           )

class_sizes = pd.read_csv(data_dir + "acssec.txt",
                          dtype={'CDSCODE': str, 'C': str, 'D': str, 'S': str},
                          usecols= lambda x: not ("Y1" in x or "Y2" in x), #Y1 corresponds to 2018-2019, Y2 corresponds to 2019-2020
                         )

ap_courses = pd.read_csv(data_dir + "ap_crsbycls.txt", 
                         dtype={"DistrictCode": str, "SchoolCode": str})

poverty_index = pd.read_csv(data_dir + "EDGE_SIDE1519_PUBSCHS1819.CSV",
                           dtype={"NCESSCH": str}
                           )

free_reduced_lunch = pd.read_excel(data_dir + "frpm2021.xlsx", 
                                   sheet_name=1, 
                                   header=1, 
                                   dtype={'County Code': str, 'District Code': str, 'School Code': str})

teacher_salary = pd.read_csv(data_dir + "salary.txt" ,
                             dtype={'CDSCODE': str, 'C': str, 'D': str, 'S': str}
                            )

### School Demographics
Information found in this section are given by demographics. We should merge these dataframes to a major dataframe after pivoting and merging by both CDSCode and demographic keys. 

In [5]:
# Disciplinary Action
suspensions = pd.read_csv(data_dir + "susexpsg.txt",
                            dtype={'CDSCode': str, 'C': str, 'D': str, 'S': str}
                           )

absences = pd.read_csv(data_dir + "chronic.txt", 
                       dtype={'CDSCode': str, 'C': str, 'D': str, 'S': str}
                      )

# Test scores
ca_science = pd.read_csv(data_dir + "casci.txt",
                         dtype={'C': str, 'D': str, 'S': str})

ca_english = pd.read_csv(data_dir + "caela.txt",
                         dtype={'C': str, 'D': str, 'S': str})

ca_math = pd.read_csv(data_dir + "camath.txt",
                      dtype={'C': str, 'D': str, 'S': str})


# Data Wrangling
Tidying up dataframes and creating new key columns for effective merging. 

In [440]:
school_cohort = school_cohort
school_cohort["CDSCode"] = (school_cohort["CountyCode"] + school_cohort["DistrictCode"] + school_cohort["SchoolCode"]).str.strip()
cols = school_cohort.columns
school_cohort = school_cohort[cols[0:1].append(cols[-1:]).append(cols[1:-1])]

In [192]:
# For school_info, create a key to connect national education information 
school_info = school_info
school_info["NCESCode"] = school_info["NCESDist"] + school_info["NCESSchool"]
cols = school_info.columns
school_info = school_info[cols[0:1].append(cols[-1:]).append(cols[1:-1])]

In [606]:
# For class sizes, set NaN = 0 and then recast all counts as ints. 
class_sizes = class_sizes.rename(columns = {"CDSCODE": "CDSCode"})
class_sizes = class_sizes.fillna(0)
cols = ['NCSEN_Y3', 'NCMEN_Y3', 'NCLEN_Y3', 'NCSMA_Y3', 'NCMMA_Y3', 'NCLMA_Y3', 'NCSSC_Y3', 'NCMSC_Y3', 'NCLSC_Y3', 'NCSSS_Y3', 'NCMSS_Y3', 'NCLSS_Y3']
class_sizes[cols] = class_sizes[cols].astype(int)

In [8]:
# For testing information, combine the individual codes to create a CDS code. 
ca_science = ca_science
ca_science["CDSCode"] = ca_science["C"] + ca_science["D"] + ca_science["S"]
cols = ca_science.columns
ca_science = ca_science[cols[-1:].append(cols[0:-1])]

In [9]:
ca_english = ca_english
ca_english["CDSCode"] = ca_english["C"] + ca_english["D"] + ca_english["S"]
cols = ca_english.columns
ca_english = ca_english[cols[-1:].append(cols[0:-1])]

In [10]:
ca_math = ca_math
ca_math["CDSCode"] = ca_math["C"] + ca_math["D"] + ca_math["S"]
cols = ca_math.columns
ca_math = ca_math[cols[-1:].append(cols[0:-1])]

In [630]:
ap_courses = ap_courses.fillna(0)
ap_courses["CDSCode"] = ap_courses["DistrictCode"] + ap_courses["SchoolCode"]
cols = ap_courses.columns
ap_courses = ap_courses[cols[-1:].append(cols[0:-1])]

In [684]:
#poverty_index = poverty_index.rename(columns = {"NCESSCH": "NCESCode", "NAME": "SchoolName"})

In [78]:
free_reduced_lunch = free_reduced_lunch
free_reduced_lunch["CDSCode"] = free_reduced_lunch["County Code"] + free_reduced_lunch["District Code"] + free_reduced_lunch["School Code"]
cols = free_reduced_lunch.columns
free_reduced_lunch = free_reduced_lunch[cols[-1:].append(cols[0:-1])]

After preliminary data wrangling, we start to join the various databases into a features matrix that is first initialized by taking sections of the school_cohort dataframe. 
The features matrix will be stored as a dataframe called school_features_data

In [462]:
# Select only the data that represents traditional high schools (both charter and non-charter). 
# The returned df will have duplicated values on ReportingCategory 
# which will need to be filtered further along "All" or everything but "All"

school_data = school_cohort.query("AggregateLevel == 'S' & " +
                                  "DASS == 'No ' & " +
                                  "CharterSchool == 'All' &" + 
                                  "SchoolCode != '0000000' & " +
                                  "SchoolCode != '0000001'")
                                
dropped_columns = school_cohort.columns.drop(
                    ["AcademicYear", "AggregateLevel", "CountyCode", "DistrictCode", "SchoolCode", 
                     "CountyName", "DistrictName", "CharterSchool", "DASS"])
school_data = school_data.dropna()[dropped_columns]
school_data.rename(columns={"Golden State Seal Merit Diploma (Rate": "Golden State Seal Merit Diploma (Rate)"}, 
                   inplace = True)
count_columns = school_data.columns[["Count" in x for x in school_data.columns]]
rate_columns = school_data.columns[["Rate" in x for x in school_data.columns]]
school_data[count_columns] = school_data[count_columns].astype(int)
school_data["CohortStudents"] = school_data["CohortStudents"].astype(int)

In [507]:
# First, select only active schools and filter for relevant information. 
school_info_short = school_info[school_info["GSserved"].str.contains("12")][["School", "CDSCode", "NCESCode", "Street", "Zip", "Magnet", "Latitude", "Longitude"]]

# Then, use Nomatim to fill in missing latitude information
geolocator = Nominatim(user_agent="lamdavid759@gmail.com")
missingGeoData = school_info_short.query("Latitude == 'No Data'").index

# Set-up cache for scraping individual results
cache_dir = 'geodata'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)
    
@checkpoint(key=lambda args, kwargs: str(args[1]) + '.pkl', work_dir=cache_dir)
def obtain_geodata(df, ind):
    time.sleep(1)
    addy = df.loc[i]["Street"] + ', ' + str(df.loc[i]["Zip"])
    try: 
        location = geolocator.geocode(addy, timeout=None)
    except GeocoderTimedOut as e: 
        print(e.message)
    else: 
        if location: 
            df.at[i, "Latitude"] = location.latitude
            df.at[i, "Longitude"] = location.longitude 

# Iterate through the entries with missing geodata
for i in missingGeoData: 
     obtain_geodata(school_info_short, i)
        
school_info_short = school_info_short[["School", "CDSCode", "NCESCode", "Magnet", "Zip", "Latitude", "Longitude"]]
school_info_short.to_csv(data_dir + "school_info_short.txt")

In [430]:
# After running the above block once, you can import school_info_shorts back in directly 
school_info_short = pd.read_csv(data_dir + "school_info_short.txt",
                                 dtype = {"CDSCode": str, "NCESCode": str, "Zip": str},
                                 index_col = 0
                                )

In [580]:
# Now merge the school_data with school_info_short
school_features_data = pd.merge(school_data, school_info_short, on = "CDSCode", how='inner')
count_columns = school_data.columns[["Count" in x for x in school_data.columns]]
rate_columns = school_data.columns[["Rate" in x for x in school_data.columns]]
selected_columns = ["CDSCode", "NCESCode", "SchoolName", "Magnet", "Zip", "Latitude", "Longitude"]
selected_columns.extend(count_columns)
selected_columns.extend(rate_columns)
school_features_data = school_features_data[selected_columns]

In [622]:
# Now merge the school_features_data with class sizes, dropping extra information for the CDS Code and Year. 
school_features_data = pd.merge(school_features_data, class_sizes, on = "CDSCode", how="left").drop(["C", "D", "S", "SARCYear"], axis=1)

# Info about the number of offered AP courses and the percentage of students who take AP courses. 
school_features_data = pd.merge(school_features_data, ap_courses, on = "CDSCode", how="left").drop(["DistrictCode", "SchoolCode", "SARCYear"], axis=1)

In [693]:
df4 = pd.merge(school_features_data, poverty_index.rename(columns = {"NCESSCH": "NCESCode"}), on="NCESCode", how = "left", indicator=True)


In [694]:
df4

,CDSCode,NCESCode,SchoolName,Magnet,Zip,Latitude,Longitude,Regular HS Diploma Graduates (Count),Met UC/CSU Grad Req's (Count),Seal of Biliteracy (Count),Golden State Seal Merit Diploma (Count),CHSPE Completer (Count),Adult Ed. HS Diploma (Count),SPED Certificate (Count),GED Completer (Count),Other Transfer (Count),Dropout (Count),Still Enrolled (Count),Regular HS Diploma Graduates (Rate),Met UC/CSU Grad Req's (Rate),Seal of Biliteracy (Rate),Golden State Seal Merit Diploma (Rate),CHSPE Completer (Rate),Adult Ed. HS Diploma (Rate),SPED Certificate (Rate),GED Completer (Rate),Other Transfer (Rate),Dropout (Rate),Still Enrolled (Rate),AVGEN_Y3,NCSEN_Y3,NCMEN_Y3,NCLEN_Y3,AVGMA_Y3,NCSMA_Y3,NCMMA_Y3,NCLMA_Y3,AVGSC_Y3,NCSSC_Y3,NCMSC_Y3,NCLSC_Y3,AVGSS_Y3,NCSSS_Y3,NCMSS_Y3,NCLSS_Y3,PSIAPC,CSNAPC,ENAPC,FPANAPC,FLNAPC,MNAPC,SNAPC,SSNAPC,ALLNAPC,NAME,IPR_EST,IPR_SE,_merge
0,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,48,48,0,13,0,0,0,0,0,2,0,96.0,100.0,0.0,27.1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9,NaN,NaN,NaN,left_only
1,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,37,37,0,6,0,0,0,0,0,7,0,84.1,100.0,0.0,16.2,0.0,0.0,0.0,0.0,0.0,15.9,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9,NaN,NaN,NaN,left_only
2,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,21,21,0,2,0,0,0,0,0,2,0,91.3,100.0,0.0,9.5,0.0,0.0,0.0,0.0,0.0,8.7,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9,NaN,NaN,NaN,left_only
3,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,54,54,0,15,0,0,0,0,0,4,0,93.1,100.0,0.0,27.8,0.0,0.0,0.0,0.0,0.0,6.9,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9,NaN,NaN,NaN,left_only
4,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,23,23,0,4,0,0,0,0,0,1,0,95.8,100.0,0.0,17.4,0.0,0.0,0.0,0.0,0.0,4.2,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13514,58727695838305,064235006930,Wheatland Union High,N,95692-9798,38.998968,-121.45497,19,7,0,0,0,0,0,0,0,0,1,95.0,36.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,23.0,12,20,0,22.0,15,16,0,23.0,6,13,0,24.0,6,16,0,16.1,0,4,0,1,2,1,4,12,Wheatland Union High,364.0,46.0,both
13515,58727695838305,064235006930,Wheatland Union High,N,95692-9798,38.998968,-121.45497,76,33,0,0,0,0,2,0,0,0,7,89.4,43.4,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,8.2,23.0,12,20,0,22.0,15,16,0,23.0,6,13,0,24.0,6,16,0,16.1,0,4,0,1,2,1,4,12,Wheatland Union High,364.0,46.0,both
13516,58727695838305,064235006930,Wheatland Union High,N,95692-9798,38.998968,-121.45497,15,1,0,0,0,0,2,0,0,0,9,57.7,6.7,0.0,0.0,0.0,0.0,7.7,0.0,0.0,0.0,34.6,23.0,12,20,0,22.0,15,16,0,23.0,6,13,0,24.0,6,16,0,16.1,0,4,0,1,2,1,4,12,Wheatland Union High,364.0,46.0,both
13517,58727695838305,064235006930,Wheatland Union High,N,95692-9798,38.998968,-121.45497,94,27,0,0,0,0,2,0,0,0,7,91.3,28.7,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,6.8,23.0,12,20,0,22.0,15,16,0,23.0,6,13,0,24.0,6,16,0,16.1,0,4,0,1,2,1,4,12,Wheatland Union High,364.0,46.0,both


In [687]:
test_schools = df4[df4["_merge"] == "left_only"]["SchoolName"].unique()

In [688]:
test_schools

array(['Envision Academy for Arts & Technology',
       'Connecting Waters Charter - East Bay',
       'Nea Community Learning Center',
       'Alameda Community Learning Center',
       'Leadership Public Schools - Hayward',
       'Silver Oak High Public Montessori Charter',
       'Impact Academy of Arts & Technology', 'Oakland Unity High',
       'Bay Area Technology', 'Lighthouse Community Charter High',
       'American Indian Public High', 'Oakland Charter High',
       'ARISE High', 'Aspire Golden State College Preparatory Academy',
       'LPS Oakland R & D Campus', 'East Bay Innovation Academy',
       'Oakland Military Institute, College Preparatory Academy',
       'Aspire Lionel Wilson College Preparatory Academy',
       'Conservatory of Vocal/Instrumental Arts High',
       'Oakland School for the Arts', 'KIPP King Collegiate High',
       'CORE Butte Charter', 'Inspire School of Arts and Sciences',
       'Ipakanni Early College Charter', 'Making Waves Academy',
       

In [689]:
n = 0
poverty_index.query(f"SchoolName == '{test_schools[n]}'")

,NCESCode,SchoolName,IPR_EST,IPR_SE
6512,060161410947,Envision Academy for Arts & Technology,144,53


In [690]:
poverty_index.query(f"NCESCode == '060161410947'")

,NCESCode,SchoolName,IPR_EST,IPR_SE
6512,060161410947,Envision Academy for Arts & Technology,144,53


In [691]:
school_features_data.query(f"SchoolName == '{test_schools[n]}'")

,CDSCode,NCESCode,SchoolName,Magnet,Zip,Latitude,Longitude,Regular HS Diploma Graduates (Count),Met UC/CSU Grad Req's (Count),Seal of Biliteracy (Count),Golden State Seal Merit Diploma (Count),CHSPE Completer (Count),Adult Ed. HS Diploma (Count),SPED Certificate (Count),GED Completer (Count),Other Transfer (Count),Dropout (Count),Still Enrolled (Count),Regular HS Diploma Graduates (Rate),Met UC/CSU Grad Req's (Rate),Seal of Biliteracy (Rate),Golden State Seal Merit Diploma (Rate),CHSPE Completer (Rate),Adult Ed. HS Diploma (Rate),SPED Certificate (Rate),GED Completer (Rate),Other Transfer (Rate),Dropout (Rate),Still Enrolled (Rate),AVGEN_Y3,NCSEN_Y3,NCMEN_Y3,NCLEN_Y3,AVGMA_Y3,NCSMA_Y3,NCMMA_Y3,NCLMA_Y3,AVGSC_Y3,NCSSC_Y3,NCMSC_Y3,NCLSC_Y3,AVGSS_Y3,NCSSS_Y3,NCMSS_Y3,NCLSS_Y3,PSIAPC,CSNAPC,ENAPC,FPANAPC,FLNAPC,MNAPC,SNAPC,SSNAPC,ALLNAPC
0,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,48,48,0,13,0,0,0,0,0,2,0,96.0,100.0,0.0,27.1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9
1,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,37,37,0,6,0,0,0,0,0,7,0,84.1,100.0,0.0,16.2,0.0,0.0,0.0,0.0,0.0,15.9,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9
2,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,21,21,0,2,0,0,0,0,0,2,0,91.3,100.0,0.0,9.5,0.0,0.0,0.0,0.0,0.0,8.7,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9
3,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,54,54,0,15,0,0,0,0,0,4,0,93.1,100.0,0.0,27.8,0.0,0.0,0.0,0.0,0.0,6.9,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9
4,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,23,23,0,4,0,0,0,0,0,1,0,95.8,100.0,0.0,17.4,0.0,0.0,0.0,0.0,0.0,4.2,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9
5,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,75,75,0,17,0,0,0,0,0,7,0,91.5,100.0,0.0,22.7,0.0,0.0,0.0,0.0,0.0,8.5,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9
6,01100170112607,069105110947,Envision Academy for Arts & Technology,N,94612-3355,37.804520,-122.26815,85,85,0,19,0,0,0,0,0,9,0,90.4,100.0,0.0,22.4,0.0,0.0,0.0,0.0,0.0,9.6,0.0,21.0,12,7,0,20.0,14,9,0,21.0,8,7,0,20.0,8,9,0,24.8,0,0,0,2,1,0,6,9


# Visualization Scripts
These are copied over from the first version, so are buggy and should be eventually deprecated.

In [ ]:
## Geospatial Figures Data Imports (Takes a while, do not run too much)
# Original data from https://github.com/OpenDataDE/State-zip-code-GeoJSON and simplified with https://mapshaper.org/ down to .5%
with open("Data Files/" + "ca_california_zip_codes_geo.min.json", "r") as response: # Original geoJSON data
    zipcodes_complex = json.load(response)
with open("Data Files/" +"ca_california_zip_codes_geo_simplified.min.json", "r") as response: # Simplified from mapshaper
    zipcodes = json.load(response)
zipcodes = rewind(zipcodes, rfc7946=False)